In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

### SPARK 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

In [3]:
from pyspark.sql.types import *

### func 

In [4]:
def list_to_set(source_lst):
    a = set()
    for i in source_lst:
        a = a | set(i)
    return list(a)   

In [5]:
set_from_list_udf = f.udf(list_to_set, ArrayType(StringType()))

In [6]:
date2 = '2021-07-04'

In [7]:
date1 = '2021-04-01'

In [8]:
input_table = 'MA_MMB_OFFER_NONTOP'
input_schema = 'SBX_TEAM_DIGITCAMP'

### dict 

In [9]:
%%time
prod_dct = hive.sql("select * from {}.{} where CRM_PRODUCT_ID is not Null".format(SBX_TEAM_DIGITCAMP, 'ma_dict_v_product_dict'))
prod_dct = prod_dct.filter("(PRODUCT_CD_MMB is not Null) and (PRODUCT_SHORT_NM is not Null)").cache()

CPU times: user 2.12 ms, sys: 2.16 ms, total: 4.28 ms
Wall time: 3.68 s


In [ ]:
prod_dct.show(3)

In [11]:
prod_dct.count()

131



дата фабрики не новее, чем дата расчета АЛС

нонтоп - состояние на текущий день
приорити - рейтинги на предыдущем расчете, заселектить одну или два дня расчета (посмотреть) - не агрегироть (максимум среднее)

брать средний потенциал вместо суммы

ma_mmb_offer_toxic




сделать юнион источников нонтоп и приорити

### Nontop 

In [17]:
def create_nontop_df(input_schema, input_table, date):
    
    nontop = hive.sql("select * from {}.{} where (INN is not Null) and (CREATE_DTTM >= timestamp('{}'))"\
                 .format(input_schema, input_table, date)).cache()
    
    nontop_need_columns = nontop.select(f.to_date(f.col('CREATE_DTTM')).alias('CREATE_DATE'),
                                        f.col("INN"),
                                        f.col("PRODUCT_ID"),
                                        f.split(f.col("SOURCES_LV2_LIST"), ',')\
                                         .alias("SOURCES_LV2_LIST_NONTOP"),
                                        f.col("COEF_PRODUCT_CD")\
                                         .alias("POTENTIAL_NONTOP")).cache()
    
    nontop_all_sources = nontop_need_columns.groupby(f.col("CREATE_DATE"),
                                                     f.col("INN"),
                                                     f.col("PRODUCT_ID")).agg(f.collect_set("SOURCES_LV2_LIST_NONTOP")\
                                                                               .alias("SOURCES_LV2_LIST_NONTOP"),
                                                                              f.avg("POTENTIAL_NONTOP")\
                                                                               .alias("POTENTIAL_NONTOP")).cache()
    
    nontop_before_end = nontop_all_sources.withColumn("SOURCES_LV2_LIST_NONTOP",
                                                      set_from_list_udf(f.col("SOURCES_LV2_LIST_NONTOP")))
    
    nontop_final = nontop_before_end.join(prod_dct,
                                          nontop_before_end.PRODUCT_ID == prod_dct.ID,
                                          how='inner')\
                                    .select(f.col("CREATE_DATE"),
                                            f.col("INN"),
                                            f.col("CRM_PRODUCT_ID").alias("CRM_PRODUCT_ID"),
                                            f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                            f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                            f.col("SOURCES_LV2_LIST_NONTOP"),
                                            f.col("POTENTIAL_NONTOP")
                                           ).cache()
        
    
    return nontop_final

In [18]:
%%time

nontop_prepr = create_nontop_df(input_schema, input_table, date1)

CPU times: user 13.7 ms, sys: 6.75 ms, total: 20.4 ms
Wall time: 271 ms


In [19]:
%%time
nontop_prepr.count()

CPU times: user 67.8 ms, sys: 42.2 ms, total: 110 ms
Wall time: 40.3 s


33296952

In [ ]:
nontop_prepr.show()

In [22]:
date1

'2021-04-30'

### Priority 

In [41]:
def create_priority_df(input_schema, input_table, date):
    
    priority = hive.sql("select * from {}.{} where inn is not Null and load_dttm >= timestamp('{}')"\
                    .format(input_schema, input_table, date)).cache()
    
    priority_need_columns = priority.select(f.to_date(f.col('load_dttm')).alias('CREATE_DATE'),
                                            f.col("INN"),
                                            f.col("CRM_PRODUCT_ID"),
                                            f.split(f.col("SOURCES_LV2_LIST"), ',')\
                                             .alias("SOURCES_LV2_LIST_PRIORITY"),
                                            f.col("offer_priority")\
                                             .alias("POTENTIAL_PRIORITY")).cache()
    
    priority_all_sources = priority_need_columns.groupby(f.col("CREATE_DATE"),
                                                         f.col("INN"),
                                                         f.col("CRM_PRODUCT_ID")).agg(f.collect_set("SOURCES_LV2_LIST_PRIORITY")\
                                                                                      .alias("SOURCES_LV2_LIST_PRIORITY"),
                                                                                     f.avg("POTENTIAL_PRIORITY")\
                                                                                      .alias("POTENTIAL_PRIORITY")).cache()
                                                         
    priority_before_end = priority_all_sources.withColumn("SOURCES_LV2_LIST_PRIORITY",
                                                      set_from_list_udf(f.col("SOURCES_LV2_LIST_PRIORITY")))
    
    priority_final = priority_before_end.join(prod_dct,
                                              priority_before_end.CRM_PRODUCT_ID == prod_dct.CRM_PRODUCT_ID,
                                              how='inner')\
                                        .select(f.col("CREATE_DATE"),
                                                f.col("INN"),
                                                priority_before_end.CRM_PRODUCT_ID.alias("CRM_PRODUCT_ID"),
                                                f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                                f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                                f.col("SOURCES_LV2_LIST_PRIORITY"),
                                                f.col("POTENTIAL_PRIORITY")
                                                ).cache()
    
    
        
    
    return priority_final

In [42]:
priority_prepr = create_priority_df(input_schema, "OFFER_PRIORITY", date1)

In [43]:
%%time
priority_prepr.count()

CPU times: user 40.6 ms, sys: 32.9 ms, total: 73.5 ms
Wall time: 4.63 s


1294125

In [ ]:
priority_prepr.show()

### toxic 

In [79]:
def create_toxic_df(input_schema, input_table, date):
    toxic = hive.sql("select * from {}.{} where inn is not Null and create_dttm >= timestamp('{}')"\
                     .format(input_schema, input_table, date)).cache()
    
    toxic_need_columns = toxic.select(f.to_date(f.col('create_dttm')).alias('CREATE_DATE'),
                                      f.col("INN"),
                                      f.col("product_id"),
                                      f.split(f.col("SOURCES_LV2_LIST"), ',')\
                                       .alias("SOURCES_LV2_LIST_TOXIC")).cache()
    
    toxic_all_sources = toxic_need_columns.groupby(f.col("CREATE_DATE"),
                                                   f.col("INN"),
                                                   f.col("product_id")).agg(f.collect_set("SOURCES_LV2_LIST_TOXIC")\
                                                                             .alias("SOURCES_LV2_LIST_TOXIC")).cache()
    
    toxic_before_end = toxic_all_sources.withColumn("SOURCES_LV2_LIST_TOXIC",
                                                    set_from_list_udf(f.col("SOURCES_LV2_LIST_TOXIC")))
    
    toxic_final = toxic_before_end.join(prod_dct,
                                        toxic_before_end.product_id == prod_dct.ID,
                                        how='inner')\
                                  .select(f.col("CREATE_DATE"),
                                          f.col("INN"),
                                          prod_dct.CRM_PRODUCT_ID.alias("CRM_PRODUCT_ID"),
                                          f.col("PRODUCT_GROUP").alias("PRODUCT_GROUP"),
                                          f.col("PRODUCT_SUBGROUP").alias("PRODUCT_SUBGROUP"),
                                          f.col("SOURCES_LV2_LIST_TOXIC")
                                         ).cache()
        
    
    return toxic_final

In [80]:
toxic_prepr = create_toxic_df(input_schema, "ma_mmb_offer_toxic", date1)

In [81]:
toxic_prepr.count()

31904724

In [ ]:
toxic_prepr.show(3)

In [86]:
toxic_prepr.select("CREATE_DATE").distinct().show()

,CREATE_DATE
0,2021-04-21
1,2021-04-20


### Joiner 

In [45]:
def join_priority_nontop(priority_final, nontop_final):
    
    result_table = priority_final.join(nontop_final,
                                   (priority_final.INN == nontop_final.INN) &
                                   (priority_final.CREATE_DATE == nontop_final.CREATE_DATE) &
                                   (priority_final.CRM_PRODUCT_ID == nontop_final.CRM_PRODUCT_ID),
                                   how='full_outer')\
                                 .select(f.when(f.isnull(priority_final.CREATE_DATE),
                                                nontop_final.CREATE_DATE).otherwise(priority_final.CREATE_DATE)\
                                                                         .alias("CREATE_DATE"),
                                         f.when(f.isnull(priority_final.INN),
                                                nontop_final.INN).otherwise(priority_final.INN)\
                                                                 .alias("INN"),
                                         f.when(f.isnull(priority_final.CRM_PRODUCT_ID),
                                                nontop_final.CRM_PRODUCT_ID).otherwise(priority_final.CRM_PRODUCT_ID)\
                                                                            .alias("CRM_PRODUCT_ID"),
                                         f.when(f.isnull(priority_final.PRODUCT_GROUP),
                                                nontop_final.PRODUCT_GROUP).otherwise(priority_final.PRODUCT_GROUP)\
                                                                           .alias("PRODUCT_GROUP"),
                                         f.when(f.isnull(priority_final.PRODUCT_SUBGROUP),
                                                nontop_final.PRODUCT_SUBGROUP).otherwise(priority_final.PRODUCT_SUBGROUP)\
                                                                              .alias("PRODUCT_SUBGROUP"),
                                         f.col("SOURCES_LV2_LIST_NONTOP"),
                                         f.col("POTENTIAL_NONTOP"),
                                         f.col("SOURCES_LV2_LIST_PRIORITY"),
                                         f.col("POTENTIAL_PRIORITY"))\
                                 .cache()
            
    
    return result_table

In [ ]:
toxic_prepr.join()

In [107]:
def join_priority_nontop_toxic(priority_final, nontop_final, toxic_final):
    
    result_table = priority_final.join(nontop_final,
                                   (priority_final.INN == nontop_final.INN) &
                                   (priority_final.CREATE_DATE == nontop_final.CREATE_DATE) &
                                   (priority_final.CRM_PRODUCT_ID == nontop_final.CRM_PRODUCT_ID),
                                   how='full_outer')\
                                 .select(f.when(f.isnull(priority_final.CREATE_DATE),
                                                nontop_final.CREATE_DATE).otherwise(priority_final.CREATE_DATE)\
                                                                         .alias("CREATE_DATE"),
                                         f.when(f.isnull(priority_final.INN),
                                                nontop_final.INN).otherwise(priority_final.INN)\
                                                                 .alias("INN"),
                                         f.when(f.isnull(priority_final.CRM_PRODUCT_ID),
                                                nontop_final.CRM_PRODUCT_ID).otherwise(priority_final.CRM_PRODUCT_ID)\
                                                                            .alias("CRM_PRODUCT_ID"),
                                         f.when(f.isnull(priority_final.PRODUCT_GROUP),
                                                nontop_final.PRODUCT_GROUP).otherwise(priority_final.PRODUCT_GROUP)\
                                                                           .alias("PRODUCT_GROUP"),
                                         f.when(f.isnull(priority_final.PRODUCT_SUBGROUP),
                                                nontop_final.PRODUCT_SUBGROUP).otherwise(priority_final.PRODUCT_SUBGROUP)\
                                                                              .alias("PRODUCT_SUBGROUP"),
                                         f.col("SOURCES_LV2_LIST_NONTOP"),
                                         f.col("POTENTIAL_NONTOP"),
                                         f.col("SOURCES_LV2_LIST_PRIORITY"),
                                         f.col("POTENTIAL_PRIORITY"))\
                                 .cache()
            
    tmp_potential_rating = result_table.join(toxic_final,
                                             (result_table.INN == toxic_final.INN) &
                                             (result_table.CREATE_DATE == toxic_final.CREATE_DATE) &
                                             (result_table.CRM_PRODUCT_ID == toxic_final.CRM_PRODUCT_ID),
                                             how='full_outer')\
                                        .select(f.when(f.isnull(result_table.CREATE_DATE),
                                                       toxic_final.CREATE_DATE).otherwise(result_table.CREATE_DATE)\
                                                                                .alias("CREATE_DATE"),
                                                f.when(f.isnull(result_table.INN),
                                                       toxic_final.INN).otherwise(result_table.INN)\
                                                                        .alias("INN"),
                                                f.when(f.isnull(result_table.CRM_PRODUCT_ID),
                                                       toxic_final.CRM_PRODUCT_ID).otherwise(result_table.CRM_PRODUCT_ID)\
                                                                                   .alias("CRM_PRODUCT_ID"),
                                                f.when(f.isnull(result_table.PRODUCT_GROUP),
                                                       toxic_final.PRODUCT_GROUP).otherwise(result_table.PRODUCT_GROUP)\
                                                                                  .alias("PRODUCT_GROUP"),
                                                f.when(f.isnull(result_table.PRODUCT_SUBGROUP),
                                                       toxic_final.PRODUCT_SUBGROUP).otherwise(result_table.PRODUCT_SUBGROUP)\
                                                                                     .alias("PRODUCT_SUBGROUP"),
                                                f.col("SOURCES_LV2_LIST_NONTOP"),
                                                f.col("POTENTIAL_NONTOP"),
                                                f.col("SOURCES_LV2_LIST_PRIORITY"),
                                                f.col("POTENTIAL_PRIORITY"),
                                                f.col("SOURCES_LV2_LIST_TOXIC"))\
                                 .cache()         
    
    return tmp_potential_rating

In [84]:
result_table = join_priority_nontop(priority_prepr, nontop_prepr)

In [85]:
result_table.count()

34564165

In [ ]:
result_table.show()

In [96]:
tmp_potential_rating = result_table.join(toxic_final,
                  (result_table.INN == toxic_final.INN) &
                  (result_table.CREATE_DATE == toxic_final.CREATE_DATE) &
                  (result_table.CRM_PRODUCT_ID == toxic_final.CRM_PRODUCT_ID),
                  how='full_outer')\
            .select(f.when(f.isnull(result_table.CREATE_DATE),
                                                toxic_final.CREATE_DATE).otherwise(result_table.CREATE_DATE)\
                                                                         .alias("CREATE_DATE"),
                                         f.when(f.isnull(result_table.INN),
                                                toxic_final.INN).otherwise(result_table.INN)\
                                                                 .alias("INN"),
                                         f.when(f.isnull(result_table.CRM_PRODUCT_ID),
                                                toxic_final.CRM_PRODUCT_ID).otherwise(result_table.CRM_PRODUCT_ID)\
                                                                            .alias("CRM_PRODUCT_ID"),
                                         f.when(f.isnull(result_table.PRODUCT_GROUP),
                                                toxic_final.PRODUCT_GROUP).otherwise(result_table.PRODUCT_GROUP)\
                                                                           .alias("PRODUCT_GROUP"),
                                         f.when(f.isnull(result_table.PRODUCT_SUBGROUP),
                                                toxic_final.PRODUCT_SUBGROUP).otherwise(result_table.PRODUCT_SUBGROUP)\
                                                                              .alias("PRODUCT_SUBGROUP"),
                                         f.col("SOURCES_LV2_LIST_NONTOP"),
                                         f.col("POTENTIAL_NONTOP"),
                                         f.col("SOURCES_LV2_LIST_PRIORITY"),
                                         f.col("POTENTIAL_PRIORITY"),
                                         f.col("SOURCES_LV2_LIST_TOXIC"))\
                                 .cache()

In [97]:
tmp_potential_rating.count()

66468722

In [ ]:
tmp_potential_rating.filter("SOURCES_LV2_LIST_PRIORITY is not Null and SOURCES_LV2_LIST_NONTOP is not null").show()

In [108]:
tmp_final = join_priority_nontop_toxic(priority_prepr, nontop_prepr, toxic_prepr)

In [109]:
tmp_final.count()

66468722

In [110]:
create_table_from_df(SBX_TEAM_DIGITCAMP, "POTENTIAL_ALL_SOURCES_TMP", tmp_final, hive)

In [ ]:
tmp_final.show()